Start by deploying LLM `Mistral 7B Instruct` to Baseten by following the documentation [here](https://www.baseten.co/library/mistral-7b-instruct/):

In [ ]:
# Initalizing Minimal Baseten Client
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Baseten configuration
BASETEN_API_KEY = os.getenv('BASETEN_API_KEY')
BASE_URL = os.getenv('BASE_URL')
MODEL_NAME = os.getenv('MODEL_NAME')

client = OpenAI(
    api_key=BASETEN_API_KEY,
    base_url=BASE_URL
)

Begin interfacing with the deployed LLM:

In [ ]:
# Testing the chat completions endpoint
response_chat = client.chat.completions.create(
        model=MODEL_NAME, # type: ignore to suppress type checking error
        messages=[
            {
                "role": "user", "content": "Write a short poem about the sea."}
        ],
        temperature=0.4,
        max_tokens=150,
    )

assistant_message = response_chat.choices[0].message.content
print(f"\nRESPONSE:\n{assistant_message}")

Moved on to zero-shot categorization model from HuggingFace:

started by installing truss prerequsite:
`pip install --upgrade truss 'pydantic>=2.0.0'`

next, I initialized the model

```bash
$ truss init cat-model
? 📦 Name this model: categorize-model
Truss categorize-model was created in ~/cat-model
```
then I setup my config.yaml by adding a descripting and requirements
```yaml
description: Zero-shot classifier for categorizing elder tech support issues
requirements:
  - torch
  - transformers
  - accelerate
```
next, I changed model.py to use the transformers library to load the model
```python
from transformers import pipeline

CATEGORIES = [
    "Password Issue",
    "App Navigation",
    "Device Setup",
    "Email Problem",
    "Security Concern",
    "Other"
]


class Model:
    def __init__(self, **kwargs):
        self.classifier = None

    def load(self):
        # Load zero-shot classifier
        try:
            self.classifier = pipeline(
                "zero-shot-classification",
                model="MoritzLaurer/deberta-v3-base-zeroshot-v2.0"
            )
        except Exception as e:
            self.classifier = None
            print(f"Failed to load classifier pipeline: {e}")

    def predict(self, input):
        # Classify
        if self.classifier is None:
            raise RuntimeError("Classifier pipeline is not loaded.")
        result = self.classifier(input, candidate_labels=CATEGORIES)

        return {
            "category": result["labels"][0],
            "confidence": round(result["scores"][0], 4)
        }
```
Finally, I deployed the model to baseten
```bash
cd .\cat-model\
truss push
```

While the categorization model was being uploaded, I realized that I could not use the OpenAI SDK for both models. To remedy this, I revised my LLM python code to invoke an API call through requests rather than the OpenAI SDK.